# Using critical sections
A [critical section](https://en.wikipedia.org/wiki/Linearizability) is a region of code that should not run in parallel. For example, the increment of a variable is not considered an [atomic operation](https://en.wikipedia.org/wiki/Linearizability), so, it should be performed using [mutual exclusion](https://en.wikipedia.org/wiki/Mutual_exclusion).

## What happens when mutial exclusion is not used in critical sections?
### Using threads

In [1]:
# Two threads that have a critical section executed in parallel without mutual exclusion.
# This code does not work!

import threading
import time

counter = 10

def task_1():
    global counter
    for i in range(1000000):
        counter += 1
        
def task_2():
    global counter
    for i in range(1000001):
        counter -= 1
        
thread_1 = threading.Thread(target=task_1)
thread_2 = threading.Thread(target=task_2)

thread_1.start()
thread_2.start()
print("(Both threads started)")

thread_1.join()
thread_2.join()
print("\nBoth threads finished")

print('counter =', counter)

(Both threads started)

Both threads finished
counter = -99559


The same example, using mutual exclusion (using a [lock](https://docs.python.org/3/library/threading.html#lock-objects)):

In [2]:
# Two threads that have a critical section executed sequentially.

import threading
import time

lock = threading.Lock()
counter = 10

def task_1():
    global counter
    for i in range(1000000):
        with lock:
            counter += 1
        
def task_2():
    global counter
    for i in range(1000001):
        with lock:
            counter -= 1
        
thread_1 = threading.Thread(target=task_1)
thread_2 = threading.Thread(target=task_2)

thread_1.start()
thread_2.start()
print("(Both threads started)")

thread_1.join()
thread_2.join()
print("\nBoth threads finished")

print('counter =', counter)

(Both threads started)

Both threads finished
counter = 9


### Using processes

In [3]:
# Two processes that have a critical section executed sequentially

import multiprocessing
import time
import ctypes

def task_1(lock, counter):
    for i in range(10000):
        lock.acquire()
        try:
            counter.value += 1
        finally:
            lock.release()
        
def task_2(lock, counter):
    for i in range(10001):
        lock.acquire()
        try:
            counter.value -= 1
        finally:
            lock.release()

lock = multiprocessing.Lock()

manager = multiprocessing.Manager()
counter = manager.Value(ctypes.c_int, 10)

process_1 = multiprocessing.Process(target=task_1, args=(lock, counter))
process_2 = multiprocessing.Process(target=task_2, args=(lock, counter))

process_1.start()
process_2.start()
print("(Both tasks started)")

process_1.join()
process_2.join()
print("\nBoth tasks finished")

print('counter =', counter.value)

(Both tasks started)

Both tasks finished
counter = 9


### Using coroutines
Notice that the lock is not needed at all.

In [ ]:
import time

def task_1(counter):
    for i in range(1000000):
        yield
        counter.value += 1
        
def task_2(lock, counter):
    for i in range(1000001):
        yield
        counter.value -= 1

t1 = task_1()
t2 = task_2()

now = time.perf_counter()  # Real time (not only user time)
t1.__next__()  # Call task_1()
    t2.__next__()  # Call task_2()
elapsed = time.perf_counter() - now
print(f"\nelapsed {elapsed:0.2f} seconds.")
print(20*0.5 + 20*0.25)
print("a =", a)

## 4. Synchronizing parallel tasks

### 4.1. Using a queue

With threads:

In [ ]:
import threading
import time
import random
import queue

queue_length = 10
q = queue.Queue(queue_length)

class Producer(threading.Thread):
    def __init__(self, iters=10):
        super(Producer,self).__init__()
        self.iters = iters

    def run(self):
        i = 0
        while i < self.iters:
            if not q.full():
                item = random.randint(1,10)
                q.put(item)
                print('Produced {} (queue length = {})'.format(item,q.qsize()))
                i += 1
                time.sleep(random.random())
        return

class Consumer(threading.Thread):
    def __init__(self, iters=10):
        super(Consumer,self).__init__()
        self.iters = iters

    def run(self):
        i = 0
        while i < self.iters:
            if not q.empty():
                item = q.get()
                print('Consumed {} (queue length = {})'.format(item,q.qsize()))
                i += 1
                time.sleep(random.random())
        return

p = Producer()
p.start()
c = Consumer()
c.start()

p.join()
c.join()

print('done')

With processes:

In [ ]:
import multiprocessing
import time
import random

class Producer():
    def __init__(self, q, iters=10):
        super(Producer,self).__init__()
        self.iters = iters
        self.q = q

    def run(self):
        i = 0
        while i < self.iters:
            if not self.q.full():
                item = random.randint(1,10)
                self.q.put(item)
                print('Produced {} (queue length = {})'.format(item, self.q.qsize()))
                i += 1
                time.sleep(random.random())
        return
    
    def start(self):
        process = multiprocessing.Process(target=self.run)
        process.start()
        return process

class Consumer():
    def __init__(self, q, iters=10):
        super(Consumer,self).__init__()
        self.iters = iters
        self.q = q

    def run(self):
        i = 0
        while i < self.iters:
            if not self.q.empty():
                item = self.q.get()
                print('Consumed {} (queue length = {})'.format(item, self.q.qsize()))
                i += 1
                time.sleep(random.random())
        return

    def start(self):
        process = multiprocessing.Process(target=self.run)
        process.start()
        return process

queue_length = 10
q = multiprocessing.Queue(queue_length)

p = Producer(q)
task_p = p.start()
c = Consumer(q)
task_c = c.start()

task_p.join()
task_c.join()

print('done')